### PSE GYM- ASU 

In [1]:
# from product_management import Product_Management, doslfp_optimizier
# main.py
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.spatial import ConvexHull
import random
from pyomo.environ import ConcreteModel, Set, Param, Block
import pyomo.environ as pyo
import pyomo.opt as opt
solver = pyo.SolverFactory('gurobi')
import math
from math import ceil
import csv
import json
import time
from math import sqrt
import copy
import ast

### ASU Simulation

In [ ]:
from ASU_opt_model import optimize_ASU
from ASU_env import ASUEnv
days_to_simulate = 3

start_day = 1
lookahead = 4
env_id = 'ASU1'
# env = ASUEnv('ASU1', lookahead, days_to_simulate)
env = ASUEnv('ASU1')
asu_optimizer = optimize_ASU('ASU1', lookahead)

env.reset()
current_day = 1
optimization_day1_cost = {}
opt_lambda = {}
opt_reward = {}

done = False
while not done:

    state = env.render()
    asu_optimizer.update_state(state)
    
    # Solve the optimization model
    objective = asu_optimizer.solve()
    day1_cost = asu_optimizer.day1_cost()
    optimization_day1_cost[current_day] = day1_cost  
    
    opt_day_lambda = asu_optimizer.extract_optimal_lambda()
    # fetch optimal hourly lambda values for the current day
    opt_lambda[current_day] = opt_day_lambda
    # based on opt_day_lambda, calculate the reward: opt_reward[current_day]
    
    day_reward = 0.0
    for hour in range(1, 25):
        # Retrieve the lambda values for this hour (assume keys in subdict are sorted correctly, e.g., 1, 2, ...).
        hourly_lambda = opt_day_lambda[hour]
        # Convert the lambda dict to a NumPy array. # For instance, if your extreme points indices are [1,2,...,n], sort the keys and build the array.
        lambda_array = np.array([hourly_lambda[k] for k in sorted(hourly_lambda.keys())], dtype=np.float32)
        opt_action = {'lambda': lambda_array}
        
        # Take a step in the environment using this optimal action.
        state, reward, done, info = env.step(opt_action)
        day_reward += reward

    opt_reward[current_day] = day_reward

    # Approaching end of current day
    if done == True:
        print("Simulation complete for total of {} days".format(current_day))
    current_day += 1

Simulation complete for total of 3 days


In [3]:
print(optimization_day1_cost)
print(opt_reward)

{1: 1044.5566829738332, 2: 861.3590746944308, 3: 805.6513215031631}
{1: -1044.5571339701135, 2: -861.3590733169953, 3: -805.6513215031631}
